In [1]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt

# Linearni, časovno invariatni (LTI) sistemi - zajem signalov

Odzivni model linarnih, časovno invariantnih (ang. *Linear, Time-Invariant, LTI*) sistemov okarakteriziramo s t. i. **prenosnimi funkcijami**, ki povezujejo signale odziva in vzbujanja sistema, v frekvenčni (*frekvenčna prenosna funkcija*, $\alpha(\omega)$) oz. časovni (*impulzna prenosna funkcija*, $h(t)$) domeni.

**Frekvenčna prenosna funkcija** $\alpha(\omega)$ torej predstavlja razmerje med (kompleksnimi) amplitudami harmonskega odziva in vzbujana opazovanega sistema:

$$
	\alpha(\omega) = \frac{X(\omega)}{F(\omega)}
$$


**Impulzna prenosna funkcija** $h(t)$ pa podaja odziv sistema na impulzno motnjo. 

Odziv na poljubno vzbujanje (signal $f(t)$) določa naslednja zveza:

$$
	x(t) = f(t) * h(t)= h(t)*f(t),
$$

kjer $*$ označuje konvolucijo.



:::{note}

Impulzna prenosna funkcija in frekvenčna prenosna funkcija sta **Fourierov par**:

$$
	h(t)=\mathcal{F}^{-1}\{\alpha(\omega)\}\qquad\textrm{ali}\qquad%
	\alpha(\omega)=\mathcal{F}\{h(t)\}.
$$

:::



## Zajem signalov za določitev prenosnih funkcij LTI sistema

Pri eksperimentalni karakterizaciji LTI sistema torej določamo razmerje med odzivom sistema in znanim signlom vzbujanja, na podlagi izmerjenih signalov $x(t)$, $f(t)$.

Namen te vaje je spoznati laboratorijsko opremo in izvesti postopek meritve signalov, potrebnih, za določitev prenosnih funkcij mehanskega LTI sistema.

:::{admonition} Meritev prenosnih funkcij jeklenega nosilca
:class: important


Z uporabo laboratorijske merilne opreme in programskega okolja LabView pripravite zajemni sistem in opravite meritve odziva (pospeška) jeklenega nosilca na impulzno vzbujanje v različnih točkah.

Delo v laboratoriju (priprava zajemnega sistema, izvedba meritve) bo potekalo **skupinsko**. 

Vsak naj izvede meritev odziva nosilca pri **vzbujanju z modalnim kladivom v točki, podani v tabeli podatkov**, ki jo najdete v e-učilnici.

:::

## Domača naloga

:::{admonition} Domača naloga
:class: warning

V okolju Jupyter Notebook pripravite kratko poročilo *(od 3 do 10 celic s kodo)* z rezultati meritev, ki ste jih izvedli na vaji, v katerem naj bodo razvidni podatki naloge. Poročilo naj vsebuje:

* prikaz (časovne vrste ter amplitudne spektre) vseh meritev, ki ste jih opravili v skupini (pri vseh podanih točkah vzbujanja),
* prikaz časovne vrste ter amplitudnega spektra vaše meritve (pri vzbujanju v točki, podani v podatkih naloge),
* prikaz frekvenčne prenosne funkcije v vaši izmerjeni točki ($i$), ki jo ocenite z $\alpha_i(\omega) = X_i(\omega) / F_i(\omega)$.

Poročilo oddajte tudi v `.pdf` obliki (glejte [navodila za oddajo domačih nalog](intro-domace-naloge)).

**Dodatek**: 
* Na podlagi izvedene meritve določite tudi impulzno prenosno funkcijo $h_i(t)$ nosilca pri vzbujanju v vaši točki ($i$).
* Simulirajte in prikažite odziv nosilca $x(t)$ na vzbujanje z naključnim signalom sile $f_i(t)$. Naključni vzbujevalni signal pripravite po lastni presoji, na primer z uporabo modula `numpy.random`. 

:::